In [2]:
# Data ingestion

from langchain_community.document_loaders import TextLoader

loader=TextLoader("speech.txt")
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='I Have a Dream" is a public speech that was delivered by American civil rights activist and Baptist minister Martin Luther King Jr. during the March on Washington for Jobs and Freedom on August 28, 1963.[2] In the speech, King called for civil and economic rights and an end to legalized racism in the United States. Delivered to over 250,000 civil rights supporters from the steps of the Lincoln Memorial in Washington, D.C., the speech was one of the most famous moments of the civil rights movement and among the most iconic speeches in American history.[3][4]\n\nBeginning with a reference to the Emancipation Proclamation, which declared millions of slaves free in 1863,[5] King said: "one hundred years later, the Negro still is not free".[6] Toward the end of the speech, King departed from his prepared text for an improvised peroration on the theme "I have a dream". In the church spirit, Mahalia Jackson lent her support from her s

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
#web based loader

from langchain_community.document_loaders import WebBaseLoader
import bs4

#load,chunk and index the conetnt of the html page. 

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    class_=("post-title","post-content","post-header"))))

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
text_documents=loader.load()

In [6]:
#pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("Internship assignment.pdf")
docs=loader.load()



In [7]:
#text splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]


[Document(metadata={'source': 'Internship assignment.pdf', 'page': 0}, page_content='Task 1: Find & Think \nObjective: Identifying 3 potential mentors/contributors and explain Why we picked them and how to \napproach them. \n1. Elizabeth Filips \n• Profile: Instagram | Y ouTube \n• Why I picked them: She teaches deep learning, creativity, and productivity through engaging \ncontent, perfect for students. \n• How to Approach: Instagram DM with an invitation to create a short study hack series for \nstudents. \n2. Kunal Kushwaha \n• Profile: Twitter | YouTube \n• Why I picked them: Active in open-source, tech education, and mentoring students in \ncoding. \n• How to Approach: Twitter DM to invite him for a session on career advice or open-source \ncontributions. \n3. Nas Daily (Nuseir Yassin) \n• Profile: Facebook | Instagram \n• Why I picked them? Engaging storytelling approach that inspires students and young \naudiences. \n• How to Approach: Email proposal for a student impact storyte

In [8]:
## vector embeddings
from langchain_community.embeddings import OpenAIEmbeddings,OllamaEmbeddings,
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(documents[:20],OllamaEmbeddings())

SyntaxError: trailing comma not allowed without surrounding parentheses (3405364581.py, line 2)

In [ ]:
# vector database
query="what and all phases in the assignment"
result=db.similarity_search()
result[0].page_content


NameError: name 'db' is not defined

In [ ]:
# Faiss vector database
from langchain_community.vectorstores import FAISS
db1=FAISS.from_documents(documents[:20],OpenAIEmbeddings())


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
query="what and all phases in the assignment"
result=db1.similarity_search(query)
result[0].page_content

NameError: name 'db1' is not defined

In [9]:
from langchain_community.llms import Ollama
llm=Ollama(model="tinyllama")
llm

C:\Users\gudas\AppData\Local\Temp\ipykernel_17412\31939611.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="tinyllama")


Ollama(model='tinyllama')

In [10]:
##Design Chat prompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(""" 
Answer the following question based only on the provided context. Think step by step before providing a detailed answer. I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
<context>
question: {input}"""
)

In [11]:
# chain introduction
#create stuff document chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [12]:
""" A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Retrievers can be created from vector stores. bhttps://docs.langchain.com/oss/python/integrations/retrievers."""

retriever=db.as_retriver()

retriever

NameError: name 'db' is not defined

In [ ]:
""" Retrieval chain: This chain takes in a user enquiry, which is then passed to the retriver to fetch relevant documets. Those documents(and origial inputs) are then passed to an llm to generate a response
https://python.langchain/docs/modules/chains/"""

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

NameError: name 'retriever' is not defined

In [ ]:
response=retrieval_chain.invoke({"input":"An attention function can be described as mapping query"})
response["answer"]

NameError: name 'retrieval_chain' is not defined